# Domain Classifier

In [147]:
conda info --envs

# conda environments:
Note: you may need to restart the kernel to use updated packages.

#
base                     C:\ProgramData\anaconda3
domain-classifier     *  C:\Users\HP\.conda\envs\domain-classifier



In [148]:
!pip install scikit-learn

In [149]:
!pip install xgboost

In [183]:
!pip install category-encoders

  Obtaining dependency information for category-encoders from https://files.pythonhosted.org/packages/1f/e2/495811f12b2e90753fff0e42a07adb0370a725de17cc23a579ac9d3ca67c/category_encoders-2.6.2-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/9.2 MB 812.7 kB/s eta 0:00:12
     - -------------------------------------- 0.2/9.2 MB 2.1 MB/s eta 0:00:05
     -- ------------------------------------- 0.5/9.2 MB 3.3 MB/s eta 0:00:03
     ------ --------------------------------- 1.5/9.2 MB 7.2 MB/s eta 0:00:02
     ------------- -------------------------- 3.1/9.2 MB 12.5 MB/s eta 0:00:01
     --------------------- ------------------ 4.9/9.2 MB 16.5 MB/s eta 0:00:01
     ----------------------------- ---------- 6.7/9.2 MB 19.4 MB/s eta 0:00:01
     ------------------------------------ --- 8.5/9.2 MB 21.6 MB/s eta 0:00:01
     

In [220]:
!pip install jupyter_contrib_nbextensions

     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     --------------------------------------- 0.1/23.5 MB 825.8 kB/s eta 0:00:29
     ---------------------------------------- 0.2/23.5 MB 2.0 MB/s eta 0:00:12
     - -------------------------------------- 0.6/23.5 MB 3.4 MB/s eta 0:00:07
     -- ------------------------------------- 1.6/23.5 MB 7.2 MB/s eta 0:00:04
     ----- ---------------------------------- 3.3/23.5 MB 12.5 MB/s eta 0:00:02
     -------- ------------------------------- 5.2/23.5 MB 16.5 MB/s eta 0:00:02
     ----------- ---------------------------- 6.9/23.5 MB 19.3 MB/s eta 0:00:01
     -------------- ------------------------- 8.7/23.5 MB 21.5 MB/s eta 0:00:01
     ----------------- --------------------- 10.4/23.5 MB 29.7 MB/s eta 0:00:01
     -------------------- ------------------ 12.2/23.5 MB 36.4 MB/s eta 0:00:01
     ----------------------- --------------- 14.0/23.5 MB 3

## Eksploracja i czyszczenie danych

In [354]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


In [370]:
bening_file_path = 'Data/CSV_benign.csv'
malware_file_path = 'Data/CSV_malware.csv'

#okazuje się, że w kolumnach "Organization" i "llc" w przypadku wielu wierszy występuje przecinek, co przesuwa dane o jedną kolumnę w prawo
bening_df = pd.read_csv(bening_file_path, low_memory=False,  quotechar='"')
malware_df = pd.read_csv(malware_file_path, converters={'llc': lambda x: x.replace(',', '')})


### zauważyłem, że część danych została przesunięta

In [364]:
#przesunięcie danych spowodowane przecinkami w tekście w kolumnie 'Organization'
#Kolejna kwestia, jest taka, że kolumny w drugim pliku (malware) są ułożone w inny sposób, co może stwarzać kolejne problemy,
#gdyż tam też mogą występowć przecinki

In [371]:
# Sprawdzenie kolumny "len" (sąsiadującej z Organization) pod kątem formatu liczb
def check_numeric_format(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

invalid_data = bening_df[~bening_df['len'].apply(check_numeric_format)]
invalid_data

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank
2553,CH,33845.0,3599,162.23.130.190,b'admin.ch.',NaN,NaN,NaN,NaN,0,0,NaN,3120.0,0,NaN,6Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n']",0,2.15413541726763,0,ch,0,0,NaN,0.0,0.0,"[('admin.ch', 100), ('theguardian.com', 68)]",0,"['adm', 'dmi', 'min']","defaultdict(<class 'int'>, {'m': 1, 'a': 1, 'i...","['ad', 'dm', 'mi', 'in']",nanAdmin,0.0,-1.0,NaN,NaN
2554,CH,33845.0,2904,162.23.128.199,b'www.admin.ch.',NaN,NaN,NaN,NaN,0,0,NaN,3120.0,1,NaN,9Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n']",0,2.289390278819448,0,ch,0,0,NaN,0.0,0.0,"[('admin.ch', 100), ('theguardian.com', 68)]",0,"['adm', 'dmi', 'min']","defaultdict(<class 'int'>, {'i': 1, 'd': 1, 'm...","['ad', 'dm', 'mi', 'in']",nanAdmin,0.0,-1.0,NaN,NaN
2555,NaN,NaN,3599,NaN,b'www.admin.ch.',NaN,NaN,NaN,NaN,0,0,NaN,3120.0,1,NaN,9Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n']",0,2.289390278819448,0,ch,0,0,NaN,0.0,0.0,"[('admin.ch', 100), ('theguardian.com', 68)]",0,"['adm', 'dmi', 'min']","defaultdict(<class 'int'>, {'i': 1, 'd': 1, 'm...","['ad', 'dm', 'mi', 'in']",nanAdmin,0.0,-1.0,NaN,NaN
3177,DE,24940.0,608,94.130.222.149,b'phpmyadmin.net.',New York,NaN,US,2002-04-02 19:34:53,0,0,"['PHPMYADMIN.NET', 'phpmyadmin.net']",17526.0,0,phpMyAdmin Project,11Admin,0,-1.0,"['p', 'h', 'p', 'm', 'y', 'a', 'd', 'm', 'i', ...",0,2.7813014714884523,6595 days 21:20:25.734252,net,0,0,"['abuse@support.gandi.net', '9957aa1cebd82bedf...",0.0,0.0,"[('php.net', 86), ('a8.net', 86)]",0,"['php', 'hpm', 'pmy', 'mya', 'yad', 'adm', 'dm...","defaultdict(<class 'int'>, {'h': 1, 'y': 1, 'i...","['ph', 'hp', 'pm', 'my', 'ya', 'ad', 'dm', 'mi...",GANDI SAS,phpmyadmin,3.0,-1.0,NaN
3178,NaN,NaN,10799,NaN,b'phpmyadmin.net.',New York,NaN,US,2002-04-02 19:34:53,0,0,"['PHPMYADMIN.NET', 'phpmyadmin.net']",17526.0,0,phpMyAdmin Project,11Admin,0,-1.0,"['p', 'h', 'p', 'm', 'y', 'a', 'd', 'm', 'i', ...",0,2.7813014714884523,6595 days 21:20:26.913209,net,0,0,"['abuse@support.gandi.net', '9957aa1cebd82bedf...",0.0,0.0,"[('php.net', 86), ('a8.net', 86)]",0,"['php', 'hpm', 'pmy', 'mya', 'yad', 'adm', 'dm...","defaultdict(<class 'int'>, {'h': 1, 'y': 1, 'i...","['ph', 'hp', 'pm', 'my', 'ya', 'ad', 'dm', 'mi...",GANDI SAS,phpmyadmin,3.0,-1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494024,US,36351.0,21599,174.37.62.109,b'camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,0.0,Domains By Proxy,LLC,8,camaro,0.0,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.375,4785 days 18:22:20.057263,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",7.6923076923076925,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'ama', 'mar', 'aro', 'ro6']","defaultdict(<class 'int'>, {'o': 1, 'r': 1, 'm...","['ca', 'am', 'ma', 'ar', 'ro', 'o6']",GoDaddy.com LLC,camaro6,2.0
494025,US,36351.0,21599,174.37.62.109,b'www.camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,1.0,Domains By Proxy,LLC,11,camaro,0.0,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.530857153109955,4785 days 18:22:21.038080,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",5.882352941176469,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'ama', 'mar', 'aro', 'ro6']","defaultdict(<class 'int'>, {'o': 1, 'r': 1, 'm...","['ca', 'am', 'ma', 'ar', 'ro', 'o6']",GoDaddy.com LLC,camaro6,2.0
494026,US,36351.0,21599,174.37.62.109,b'www.camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,1.0,Domains By Proxy,LLC,11,camaro,0.0,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.530857153109955,4785 days 18:22:22.040938,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsb

In [372]:
# Usuń wartości w kolumnie 'len' w tych wierszach
bening_df.loc[invalid_data.index, 'len'] = None

# Przesuń dane w lewo w kolumnie 'len' w tych wierszach
bening_df.loc[invalid_data.index, 'len':] = bening_df.loc[invalid_data.index, 'len':].shift(-1, axis=1)


In [373]:
# Ponowne Sprawdzenie kolumny "len" pod kątem formatu liczb

invalid_data_len = bening_df[~bening_df['len'].apply(check_numeric_format)]
invalid_data_len

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank
249586,US,13335.0,124,104.27.189.217,b'tecadmin.net.',Arizona,NaN,US,2013-02-08 03:51:16,0,0,TECADMIN.NET,17347.0,0,Domains By Proxy,9Admin,0,-1.0,"['t', 'e', 'c', 'a', 'd', 'm', 'i', 'n']",0,2.8177111123931664,2635 days 22:17:09.319432,net,0,0,"['abuse@godaddy.com', 'TECADMIN.NET@domainsbyp...",0.0,0,"[('php.net', 86), ('a8.net', 86)]",0,"['tec', 'eca', 'cad', 'adm', 'dmi', 'min']","defaultdict(<class 'int'>, {'t': 1, 'e': 1, 'i...","['te', 'ec', 'ca', 'ad', 'dm', 'mi', 'in']",GoDaddy.com LLC,tecadmin,2.0,-1.0,NaN
249587,US,13335.0,124,104.27.189.217,b'tecadmin.net.',Arizona,NaN,US,2013-02-08 03:51:16,0,0,TECADMIN.NET,17347.0,0,Domains By Proxy,9Admin,0,-1.0,"['t', 'e', 'c', 'a', 'd', 'm', 'i', 'n']",0,2.8177111123931664,2635 days 22:17:10.208094,net,0,0,"['abuse@godaddy.com', 'TECADMIN.NET@domainsbyp...",0.0,0,"[('php.net', 86), ('a8.net', 86)]",0,"['tec', 'eca', 'cad', 'adm', 'dmi', 'min']","defaultdict(<class 'int'>, {'t': 1, 'e': 1, 'i...","['te', 'ec', 'ca', 'ad', 'dm', 'mi', 'in']",GoDaddy.com LLC,tecadmin,2.0,-1.0,NaN
415242,US,29802.0,3599,23.111.129.74,b'theadminzone.com.',CA,NaN,US,2004-01-16 04:32:19,0,0,"['THEADMINZONE.COM', 'theadminzone.com']",41104.0,0,Threadloom,13Admin,0,-1.0,"['t', 'h', 'e', 'a', 'd', 'm', 'i', 'n', 'z', ...",0,3.1080982013610083,5949 days 13:13:22.492705,com,0,0,"['registrar-abuse@google.com', 'nph9md7zq8tgsn...",0.0,0,"[('one.com', 90), ('google.com', 86)]",0,"['the', 'hea', 'ead', 'adm', 'dmi', 'min', 'in...","defaultdict(<class 'int'>, {'t': 1, 'h': 1, 'i...","['th', 'he', 'ea', 'ad', 'dm', 'mi', 'in', 'nz...",Google LLC,theadminzone,4.0,-1.0,NaN
415243,US,29802.0,3599,23.111.129.74,b'theadminzone.com.',CA,NaN,US,2004-01-16 04:32:19,0,0,"['THEADMINZONE.COM', 'theadminzone.com']",41104.0,0,Threadloom,13Admin,0,-1.0,"['t', 'h', 'e', 'a', 'd', 'm', 'i', 'n', 'z', ...",0,3.1080982013610083,5949 days 13:13:23.689544,com,0,0,"['registrar-abuse@google.com', 'nph9md7zq8tgsn...",0.0,0,"[('one.com', 90), ('google.com', 86)]",0,"['the', 'hea', 'ead', 'adm', 'dmi', 'min', 'in...","defaultdict(<class 'int'>, {'t': 1, 'h': 1, 'i...","['th', 'he', 'ea', 'ad', 'dm', 'mi', 'in', 'nz...",Google LLC,theadminzone,4.0,-1.0,NaN


In [374]:
bening_df = bening_df.drop(invalid_data_len.index)

In [ ]:
#przesunięcie danych spowodowane przecinkami w kolumnie "sld".
#co więcej patrząc na plik źródłowy dane z ostatniej kolumny w wierszach, w których występuje wspomniany przecinek,
#zostały zagubione.
#Kolejna kwestia, jest taka, że kolumny w drugim pliku (malware) są ułożone w inny sposób, co może stwarzać kolejne problemy,
#gdyż tam też mogą występowć przecinki


In [375]:
# Sprawdzenie kolumny "Name_Server_Count" (sąsiaduje z sld) pod kątem formatu liczb
invalid_data_sname = bening_df[~bening_df['Name_Server_Count'].apply(check_numeric_format)]
invalid_data_sname

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank
16384,CA,13768.0,21599,64.77.110.11,b'rheumatology.org.',GA,NaN,US,1995-10-07 04:00:00,0,0,RHEUMATOLOGY.ORG,126025.0,0,American College of Rheumatology,13,ology,0,-1.0,"['r', 'h', 'e', 'u', 'm', 'a', 't', 'o', 'l', ...",0,3.261944355207162,8965 days 19:51:21.202151,org,0,0,abuse@web.com,0.0,0,"[('w3.org', 86), ('npr.org', 86)]",0,"['rhe', 'heu', 'eum', 'uma', 'mat', 'ato', 'to...","defaultdict(<class 'int'>, {'h': 1, 'e': 1, 'o...","['rh', 'he', 'eu', 'um', 'ma', 'at', 'to', 'ol...",Network Solutions,LLC,rheumatology,4.0
16387,US,20473.0,599,108.61.19.13,b'furtdsolinopv.com.',NaN,NaN,NaN,2018-03-12 18:25:52,0,0,FURTDSOLINOPV.COM,8923099.0,0,NaN,14,lino,0,-1.0,"['f', 'u', 'r', 't', 'd', 's', 'o', 'l', 'i', ...",0,3.3925438561963475,773 days 5:25:34.818541,com,0,0,abuse@web.com,0.0,0,"[('twitter.com', 86), ('google.com', 86)]",0,"['fur', 'urt', 'rtd', 'tds', 'dso', 'sol', 'ol...","defaultdict(<class 'int'>, {'t': 1, 'o': 2, 'r...","['fu', 'ur', 'rt', 'td', 'ds', 'so', 'ol', 'li...",SNAPNAMES 46,LLC,furtdsolinopv,2.0
16388,NaN,NaN,599,NaN,b'ww1.furtdsolinopv.com.',NaN,NaN,NaN,2018-03-12 18:25:52,0,0,FURTDSOLINOPV.COM,8923099.0,1,NaN,17,lino,0,-1.0,"['f', 'u', 'r', 't', 'd', 's', 'o', 'l', 'i', ...",0,3.611729732941497,773 days 5:25:35.778178,com,0,0,abuse@web.com,4.3478260869565215,0,"[('twitter.com', 86), ('google.com', 86)]",0,"['fur', 'urt', 'rtd', 'tds', 'dso', 'sol', 'ol...","defaultdict(<class 'int'>, {'t': 1, 'o': 2, 'r...","['fu', 'ur', 'rt', 'td', 'ds', 'so', 'ol', 'li...",SNAPNAMES 46,LLC,furtdsolinopv,2.0
16389,DE,47846.0,599,91.195.240.210,b'ww1.furtdsolinopv.com.',NaN,NaN,NaN,2018-03-12 18:25:52,0,0,FURTDSOLINOPV.COM,8923099.0,1,NaN,17,lino,0,-1.0,"['f', 'u', 'r', 't', 'd', 's', 'o', 'l', 'i', ...",0,3.611729732941497,773 days 5:25:36.701658,com,0,0,abuse@web.com,4.3478260869565215,0,"[('twitter.com', 86), ('google.com', 86)]",0,"['fur', 'urt', 'rtd', 'tds', 'dso', 'sol', 'ol...","defaultdict(<class 'int'>, {'t': 1, 'o': 2, 'r...","['fu', 'ur', 'rt', 'td', 'ds', 'so', 'ol', 'li...",SNAPNAMES 46,LLC,furtdsolinopv,2.0
16397,SG,135161.0,3599,150.95.20.195,b'absc.co.th.',NaN,NaN,NaN,NaN,0,0,NaN,9525164.0,0,NaN,5,bsc,0,-1.0,"['a', 'b', 's', 'c']",0,1.8575424759098895,0,co.th,0,0,staff@thnic.co.th,0.0,0,"[('t.co', 86), ('telegraph.co.uk', 86)]",0,"['abs', 'bsc']","defaultdict(<class 'int'>, {'a': 1, 's': 1, 'c...","['ab', 'bs', 'sc']",T.H.NIC Co.,Ltd.,absc,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494098,VG,40034.0,7199,208.91.197.27,b'umbrellainvestment.com.',NaN,NaN,NaN,2006-06-20 20:29:51,0,0,UMBRELLAINVESTMENT.COM,-1.0,0.0,NaN,19,investment,0,-1.0,"['u', 'm', 'b', 'r', 'e', 'l', 'l', 'a', 'i', ...",0,3.3530425126221624,5064 days 3:48:50.266140,com,0,0,abuse@web.com,0.0,0.0,"[('t.co', 90), ('facebook.com', 86)]",0,"['umb', 'mbr', 'bre', 'rel', 'ell', 'lla', 'la...","defaultdict(<class 'int'>, {'e': 3, 't': 2, 'r...","['um', 'mb', 'br', 're', 'el', 'll', 'la', 'ai...",Network Solutions,LLC,umbrellainvestment,2.0
494101,CN,37963.0,599,119.23.87.138,b'21cbr.com.',Guang Dong,NaN,CN,2004-06-21 15:42:03,0,0,"['21CBR.COM', '21cbr.com']",-1.0,0.0,NaN,6,r,0,-1.0,"['2', '1', 'c', 'b', 'r']",0,2.15413541726763,5793 days 8:36:41.882747,com,0,0,DomainAbuse@service.aliyun.com,18.181818181818183,0.0,"[('qq.com', 86), ('vk.com', 86)]",0,"['21c', '1cb', 'cbr']","defaultdict(<class 'int'>, {'2': 1, '1': 1, 'b...","['21', '1c', 'cb', 'br']",Alibaba Cloud Computing (Beijing) Co.,Ltd.,21cbr,3.0
494102,NaN,NaN,599,NaN,b'21cbr.com.',Guang Dong,NaN,CN,2004-06-21 15:42:03,0,0,"['21CBR.

In [378]:
# Usuń wartości w kolumnie 'sld' w tych wierszach
bening_df.loc[invalid_data_sname.index, 'sld'] = None

# Przesuń dane w lewo w kolumnie 'len' w tych wierszach
bening_df.loc[invalid_data_sname.index, 'sld':] = bening_df.loc[invalid_data_sname.index, 'sld':].shift(-1, axis=1)


In [380]:
# Ponowne Sprawdzenie kolumny "Name_Server_Count" pod kątem formatu liczb
invalid_data_sname = bening_df[~bening_df['Name_Server_Count'].apply(check_numeric_format)]
invalid_data_sname

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank


In [385]:
#sprawdzę dla sąsiadującej kolumny z 'sld'
malware_df['Registrant_Name'].unique()

array([nan, 'linkgrowth media', 'Registration Private', 'Avichy Margalit',
       'fajar yuniar', 'Namecheap Namecheap', 'REDACTED FOR PRIVACY',
       'EN POINTE MARKETING PTY LIMITED', 'WZ', 'MAKY GROUP PTY LTD',
       'PAPACHRISTOU PLEUMJIT', 'FAULKS GARY JOHN',
       'Eagle and Cross Marketing', 'YCS', 'HEIDI WEITJENS',
       '2012 PTY LTD', 'QUAID JACK', 'C.T CRISPIN & K.J CRISPIN',
       "['yossi seliktar', 'Domain The Net']",
       "['Sylvain Belleau', 'Domine Vertisoft']", 'Milen Radumilo',
       'Andrzej Lukasik', "['Patrick Bell', 'REDACTED FOR PRIVACY']",
       "['Caviro Soc. Coop. Agricola', 'Massimo Emiliani']",
       'Justin Yoshimura', 'Travis Phipps', 'won young oh',
       'Steven Sikes',
       "['Cayenne Creative Incorporated', 'Carl Poirier']",
       'changeip operations', 'Tomorrow Pictures Inc',
       'WhoisGuard Protected', 'Alipasha Amidi', 'arif gursel',
       'JGL CONTRACTING PTY LTD',
       'BETTAWAY TAXI TRUCKS (QLD) PTY LIMITED', 'A1 Security Sy

In [388]:
#i dla sąsiada 'Organization'
malware_df['char_distribution'].dtype

dtype('O')

In [ ]:
#wydaje się, że w malware_df tego problemu nie ma

In [392]:
bening_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494131 entries, 0 to 494134
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Country             385190 non-null  object 
 1   ASN                 385245 non-null  float64
 2   TTL                 494131 non-null  int64  
 3   IP                  386436 non-null  object 
 4   Domain              494131 non-null  object 
 5   State               217501 non-null  object 
 6   Registrant_Name     13637 non-null   object 
 7   Country.1           254089 non-null  object 
 8   Creation_Date_Time  362218 non-null  object 
 9   hex_32              494008 non-null  object 
 10  hex_8               494131 non-null  object 
 11  Domain_Name         397434 non-null  object 
 12  Alexa_Rank          440284 non-null  object 
 13  subdomain           494115 non-null  object 
 14  Organization        201742 non-null  object 
 15  len                 493773 non-null  ob

In [391]:
bening_df.head()

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank
0,US,15169.0,46,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0
1,NaN,NaN,299,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0
2,NaN,NaN,38,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0
3,US,15169.0,299,172.217.164.228,b'www.google.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,NaN,com,0,0,NaN,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",NaN,google,NaN,NaN
4,NaN,NaN,96,NaN,b'facebook.com.',CA,NaN,US,1997-03-29 05:00:00,0,0,FACEBOOK.COM,4.0,0,Facebook inc.,9,facebook,0.0,-1.0,"['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k']",0.0,2.5954888901709436,8426 days 10:53:52.505111,com,0,0,"['abusecomplaints@registrarsafe.com', 'domain@...",0.0,0,"[('facebook.com', 100), ('qq.com', 86)]",0,"['fac', 'ace', 'ceb', 'ebo', 'boo', 'ook']","defaultdict(<class 'int'>, {'e': 1, 'c': 1, 'o...","['fa', 'ac', 'ce', 'eb', 'bo', 'oo', 'ok']",RegistrarSafe LLC,facebook,4.0,-1.0


Widać, że wiele zmiennych w zestawie bening ma błędnie przypisany typ

In [393]:
malware_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             4053 non-null   object 
 1   ASN                 4055 non-null   float64
 2   TTL                 4999 non-null   object 
 3   IP                  4098 non-null   object 
 4   Domain              4999 non-null   object 
 5   oc_8                4999 non-null   int64  
 6   longest_word        4999 non-null   object 
 7   Domain_Name         3968 non-null   object 
 8   dec_32              4999 non-null   int64  
 9   subdomain           4999 non-null   int64  
 10  sld                 4999 non-null   object 
 11  Registrant_Name     73 non-null     object 
 12  hex_32              4999 non-null   int64  
 13  Domain_Age          4428 non-null   object 
 14  entropy             4999 non-null   float64
 15  Name_Server_Count   4428 non-null   float64
 16  Emails

In [394]:
malware_df.head()

,Country,ASN,TTL,IP,Domain,oc_8,longest_word,Domain_Name,dec_32,subdomain,sld,Registrant_Name,hex_32,Domain_Age,entropy,Name_Server_Count,Emails,Creation_Date_Time,Organization,char_distribution,2gram,State,oc_32,Country.1,Page_Rank,hex_8,dec_8,1gram,len,typos,tld,shortened,Registrar,obfuscate_at_sign,Alexa_Rank,3gram,numeric_percentage,puny_coded
0,US,26347.0,14399,67.205.14.183,b'dicrophani.com.',0,hani,DICROPHANI.COM,0,0,dicrophani,NaN,0,4277 days 21:07:56.450015,2.963120,6.0,"['abuse@name.com', 'roxics@gmail.com']",2008-08-04 04:20:05,Scape Films,"defaultdict(<class 'int'>, {'d': 1, 'n': 1, 'a...","['di', 'ic', 'cr', 'ro', 'op', 'ph', 'ha', 'an...",MI,0,US,-1.0,0,0,"['d', 'i', 'c', 'r', 'o', 'p', 'h', 'a', 'n', ...",11,"[('apple.com', 86), ('adobe.com', 86)]",com,-1.0,Name.com inc.,0,-1.0,"['dic', 'icr', 'cro', 'rop', 'oph', 'pha', 'ha...",0.000000,0
1,US,46606.0,14399,70.40.220.126,b'dionneg.com.',0,dio,DIONNEG.COM,0,0,dionneg,NaN,0,4596 days 7:06:33.492321,2.375000,2.0,"['support@bluehost.com', 'GORDONDN@HOTMAIL.COM...",2007-09-20 18:21:29,NaN,"defaultdict(<class 'int'>, {'d': 1, 'n': 2, 'g...","['di', 'io', 'on', 'nn', 'ne', 'eg']",CALIFORNIA,0,US,-1.0,0,0,"['d', 'i', 'o', 'n', 'n', 'e', 'g']",8,"[('g.co', 90), ('qq.com', 86)]",com,-1.0,FastDomain Inc.,0,-1.0,"['dio', 'ion', 'onn', 'nne', 'neg']",0.000000,0
2,US,16509.0,3599,15.164.151.80,b'vipprojects.cn.',0,projects,vipprojects.cn,0,0,vipprojects,NaN,0,2457 days 14:55:14.742716,3.119549,2.0,work@marksmile.com,2013-07-29 10:32:49,NaN,"defaultdict(<class 'int'>, {'v': 1, 'c': 1, 'j...","['vi', 'ip', 'pp', 'pr', 'ro', 'oj', 'je', 'ec...",NaN,0,NaN,-1.0,0,0,"['v', 'i', 'p', 'p', 'r', 'o', 'j', 'e', 'c', ...",12,"[('4.cn', 86), ('360.cn', 86)]",cn,-1.0,杭州名商网络有限公司,0,-1.0,"['vip', 'ipp', 'ppr', 'pro', 'roj', 'oje', 'je...",0.000000,0
3,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,hhj3,NaN,0,590 days 10:04:04.890200,1.457542,2.0,yedaofeng0022@126.com,2018-09-08 15:24:00,NaN,"defaultdict(<class 'int'>, {'h': 2, '3': 1, 'j...","['hh', 'hj', 'j3']",NaN,0,NaN,-1.0,0,0,"['h', 'h', 'j', '3']",5,"[('miitbeian.gov.cn', 86), ('miibeian.gov.cn',...",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"['hhj', 'hj3']",11.111111,0
4,US,395954.0,599,23.110.187.67,b'hhj3.cn.',0,hh,hhj3.cn,0,0,hhj3,NaN,0,590 days 10:04:06.211003,1.457542,2.0,yedaofeng0022@126.com,2018-09-08 15:24:00,NaN,"defaultdict(<class 'int'>, {'h': 2, '3': 1, 'j...","['hh', 'hj', 'j3']",NaN,0,NaN,-1.0,0,0,"['h', 'h', 'j', '3']",5,"[('miitbeian.gov.cn', 86), ('miibeian.gov.cn',...",cn,-1.0,成都西维数码科技有限公司,0,-1.0,"['hhj', 'hj3']",11.111111,0


W zestawie malware przypisanie typów wygląda lepiej.
Spójrzmy na braki danych.

In [395]:
ben_nans = bening_df.isna().sum().sort_values(ascending=False)
ben_nans_percent = (bening_df.isna().sum() / bening_df.isna().count()).sort_values(ascending=False)
mal_nans = malware_df.isna().sum().sort_values(ascending=False)
mal_nans_percent = (malware_df.isna().sum() / malware_df.isna().count()).sort_values(ascending=False)

missing_data = pd.concat([ben_nans, ben_nans_percent, mal_nans, mal_nans_percent], axis=1, keys=['ben_nans', 'ben_nans_percent', 'mal_nans', 'mal_nans_percent'])
print(missing_data[missing_data["ben_nans"] > 0])


                    ben_nans  ben_nans_percent  mal_nans  mal_nans_percent
Registrant_Name       480494          0.972402      4926          0.985397
Organization          292389          0.591724      2718          0.543709
State                 276630          0.559831      2367          0.473495
Country.1             240042          0.485786      2109          0.421884
Emails                171092          0.346248      1622          0.324465
Page_Rank             153511          0.310669       571          0.114223
Registrar             139937          0.283198      1306          0.261252
Creation_Date_Time    131913          0.266960      1175          0.235047
Country               108941          0.220470       946          0.189238
ASN                   108886          0.220359       944          0.188838
IP                    107695          0.217948       901          0.180236
Domain_Name            96697          0.195691      1031          0.206241
Name_Server_Count      54

Jak widać powyżej braki danych dotyczą tych samych cech. <br>
Usunę zmiennę, które mają powyzej 30% utraty danych.

In [396]:
#Nadawanie labeli 
bening_df['domain_classifier'] = 0
malware_df['domain_classifier'] = 1

In [397]:
df = pd.concat([bening_df, malware_df], axis=0, ignore_index=True)

In [398]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499130 entries, 0 to 499129
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Country             389243 non-null  object 
 1   ASN                 389300 non-null  float64
 2   TTL                 499130 non-null  object 
 3   IP                  390534 non-null  object 
 4   Domain              499130 non-null  object 
 5   State               220133 non-null  object 
 6   Registrant_Name     13710 non-null   object 
 7   Country.1           256979 non-null  object 
 8   Creation_Date_Time  366042 non-null  object 
 9   hex_32              499007 non-null  object 
 10  hex_8               499130 non-null  object 
 11  Domain_Name         401402 non-null  object 
 12  Alexa_Rank          444712 non-null  object 
 13  subdomain           499114 non-null  object 
 14  Organization        204023 non-null  object 
 15  len                 498772 non-nul

In [399]:
#sprawdzę teraz unikalne wartości dla każdej zmiennej
unique_value_counts = []

for column in df.columns:
    unique_values = df[column].nunique()
    unique_value_counts.append((column, unique_values))

unique_value_counts_df = pd.DataFrame(unique_value_counts, columns=['Kolumna', 'Liczba unikalnych wartości'])

unique_value_counts_df = unique_value_counts_df.sort_values(by='Liczba unikalnych wartości')

print(unique_value_counts_df)

               Kolumna  Liczba unikalnych wartości
38   domain_classifier                           2
18           shortened                           7
37           Page_Rank                          10
10               hex_8                          20
24               dec_8                          47
28          puny_coded                          86
9               hex_32                         123
15                 len                         127
25              dec_32                         142
13           subdomain                         185
36   Name_Server_Count                         265
17               oc_32                         277
0              Country                         284
20   obfuscate_at_sign                         357
27  numeric_percentage                         416
30                oc_8                         443
23                 tld                        1300
7            Country.1                        1619
34           Registrar         

Cechy mają wiele wartości unikalnych. Ciekawie wygląda liczba krajów w zmiennej Country (284), a jeszcze ciekawiej zmiennej Country1 (1619). 

In [400]:
# Określenie progu procentowego dla brakujących danych (np. 30%)
threshold_percentage = 0.3

# Obliczenie liczby brakujących danych w każdej kolumnie
nans = df.isna().sum()

# Obliczenie procentu brakujących danych w każdej kolumnie
nans_percent = nans / len(df)

columns_to_drop = nans_percent >= threshold_percentage
print("Kolumny do usunięcia:")
print(columns_to_drop[columns_to_drop == True].index)

df_clean = df.drop(columns=columns_to_drop[columns_to_drop == True].index)

Kolumny do usunięcia:
Index(['State', 'Registrant_Name', 'Country.1', 'Organization', 'Emails',
       'Page_Rank'],
      dtype='object')


In [401]:
df_clean.head()

,Country,ASN,TTL,IP,Domain,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,domain_classifier
0,US,15169.0,46,172.217.1.174,b'google.com.',1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,0
1,NaN,NaN,299,NaN,b'google.com.',1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,0
2,NaN,NaN,38,NaN,b'www.google.com.',1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,0
3,US,15169.0,299,172.217.164.228,b'www.google.com.',NaN,0,0,NaN,NaN,1,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,NaN,com,0,0,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",NaN,google,NaN,0
4,NaN,NaN,96,NaN,b'facebook.com.',1997-03-29 05:00:00,0,0,FACEBOOK.COM,4.0,0,9,facebook,0.0,-1.0,"['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k']",0.0,2.5954888901709436,8426 days 10:53:52.505111,com,0,0,0.0,0,"[('facebook.com', 100), ('qq.com', 86)]",0,"['fac', 'ace', 'ceb', 'ebo', 'boo', 'ook']","defaultdict(<class 'int'>, {'e': 1, 'c': 1, 'o...","['fa', 'ac', 'ce', 'eb', 'bo', 'oo', 'ok']",RegistrarSafe LLC,facebook,4.0,0


## Podział danych na test i trening i budowa pipelinu

In [404]:
#podział danych, użyto argumentu "stratify"
X = df_clean.drop(columns=['domain_classifier'])
y = df_clean['domain_classifier']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
categorical = ['sld', 'Country', 'Registrar', 'longest' 'word', 'tld']

In [ ]:
numerical = ASN, TTL, Alexa_Rank, len, shortened, obfuscate_at_sign, entropy, Domain_Age, numeric_percentage, puny_coded, Name_Server_Count

In [402]:
#widać, że nie wszystkie błędy we wczytywaniu danych udało się naprawić
unique_values = df_clean['oc_32'].unique()[:20]
print(unique_values) 

['0.0' "['a', 'd', 'm', 'i', 'n']"
 "['p', 'h', 'p', 'm', 'y', 'a', 'd', 'm', 'i', 'n']"
 "['e', 'u', 's', 'k', 'a', 'd', 'i']" '0' 'tapas' 'no' 'industry'
 "['u', 's', 'e', 'r', 'm', 'd']" "['m', 'y', 'd', 'e', 'v', 'i', 'l']"
 '1' 'trans' 'maven' 'swagger'
 "['n', 'n', 'a', 'd', 'm', 'i', 'n', 'i', 's', 't', 'r', 'a', 't', 'i', 'e']"
 'forecast' "['p', 'h', 'p', 'n', 'e', 't']" 'sourceforge'
 "['a', 'd', 'm', 'i', 'n', 'e', 'r']" 'wow']


In [403]:
df_clean[df_clean['oc_32'] == 'swagger']

,Country,ASN,TTL,IP,Domain,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,domain_classifier
22505,US,16509.0,59,13.225.224.87,b'swagger.io.',NaN,2014-07-19 09:07:18,0,0,SWAGGER.IO,15921.0,NaN,8,swagger,0,-1.0,"['s', 'w', 'a', 'g', 'g', 'e', 'r']",0,2.375,2105 days 17:36:34.881802,io,0,NaN,0.0,0,"[('blogger.com', 57), ('sagepub.com', 57)]",0,"['swa', 'wag', 'agg', 'gge', 'ger']","defaultdict(<class 'int'>, {'e': 1, 'r': 1, 'a...","['sw', 'wa', 'ag', 'gg', 'ge', 'er']",swagger,4.0,0
22506,US,16509.0,59,13.225.224.87,b'swagger.io.',NaN,2014-07-19 09:07:18,0,0,SWAGGER.IO,15921.0,NaN,8,swagger,0,-1.0,"['s', 'w', 'a', 'g', 'g', 'e', 'r']",0,2.375,2105 days 17:36:37.968733,io,0,NaN,0.0,0,"[('blogger.com', 57), ('sagepub.com', 57)]",0,"['swa', 'wag', 'agg', 'gge', 'ger']","defaultdict(<class 'int'>, {'e': 1, 'r': 1, 'a...","['sw', 'wa', 'ag', 'gg', 'ge', 'er']",swagger,4.0,0


In [414]:
df_clean['obfuscate_at_sign'].unique()[:20] #widać że wiele danych jest nie w swojej kolumnie

array(['0.0', '0', '6595 days 21:20:25.734252',
       '6595 days 21:20:26.913209', '6595 days 21:20:28.057126', nan,
       '1976 days 1:10:56.635849', '1976 days 1:10:57.616898',
       '1976 days 1:10:58.729397', '1976 days 1:10:59.709627',
       '1976 days 1:11:01.234285', "['t', 'a', 'p', 'a', 's']",
       "['n', 'o', '-', 'i', 'p']",
       "['i', 'n', 'd', 'u', 's', 't', 'r', 'y']",
       '1976 days 7:29:39.862707', '1976 days 7:29:41.435882',
       '2060 days 14:23:30.548447', '5714 days 4:10:11.897025',
       "['t', 'r', 'a', 'n', 's', 'p', 'o', 'r', 't', 'r']",
       "['n', 'e', 'w', 's', 'm', 'a', 'v', 'e', 'n']"], dtype=object)

In [184]:
import category_encoders as ce

# Tworzenie obiektu TargetEncoder
target_encoder = ce.TargetEncoder(cols=["Country"])

# Dopasowanie i przekształcenie danych treningowych
X_train_encoded = target_encoder.fit_transform(X_train, y_train)

# Przekształcenie danych testowych
X_test_encoded = target_encoder.transform(X_test)


In [115]:
# #undersampling
# count_benign = sum(y_train == 0)
# count_malware = sum(y_train == 1)
# train_data = pd.concat([X_train, y_train], axis=1)
# # Podział na dane bening i malware
# bening = train_data[train_data['domain_classifier'] == 0]
# malware = train_data[train_data['domain_classifier'] == 1]
# bening_undersampled = bening.sample(count_malware, replace=False, random_state=42)
# # Połącz dane malware i bening
# combined_data = pd.concat([malware, bening_undersampled])

# # Podziel dane na cechy (X) i etykiety (y)
# X_train_us = combined_data.drop('domain_classifier', axis=1)  
# y_train_us = combined_data['domain_classifier'] 

In [185]:
def undersample(X, y):
    count_benign = sum(y == 0)
    count_malware = sum(y == 1)
    train_data = pd.concat([X, y], axis=1)
    bening = train_data[train_data['domain_classifier'] == 0]
    malware = train_data[train_data['domain_classifier'] == 1]
    bening_undersampled = bening.sample(count_malware, replace=False, random_state=42)
    combined_data = pd.concat([malware, bening_undersampled])
    X = combined_data.drop('domain_classifier', axis=1)
    y = combined_data['domain_classifier']
    return X, y


In [186]:
X_train_us, y_train_us = undersample(X_train, y_train)

In [188]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score

# Tworzenie i trenowanie modelu XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train_us, y_train_us)

# Predykcja na zbiorze testowym
y_pred = xgb_model.predict(X_test)

# Obliczenie metryk jakości
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Dokładność (Accuracy):", accuracy)
print("Precyzja (Precision):", precision)
print("F1 Score:", f1)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:TTL: object, IP: object, Domain: object, State: object, Registrant_Name: object, Country.1: object, Creation_Date_Time: object, hex_32: object, hex_8: object, Domain_Name: object, Alexa_Rank: object, subdomain: object, Organization: object, len: object, longest_word: object, oc_32: object, shortened: object, 1gram: object, obfuscate_at_sign: object, entropy: object, Domain_Age: object, tld: object, dec_8: object, dec_32: object, Emails: object, numeric_percentage: object, puny_coded: object, typos: object, oc_8: object, 3gram: object, char_distribution: object, 2gram: object, Registrar: object, sld: object, Name_Server_Count: object